<img src='./img/LogoWekeo_Copernicus_RGB_0.png' alt='' align='centre' width='30%'></img>

# SLSTR spatial plotting, quality control and data interrogation

    Version: 3.0
    Date:    13/07/2020
    Author:  Ben Loveday (InnoFlair, Plymouth Marine Laboratory) and Hayley Evers-King (EUMETSAT)
    Credit:  This code was developed for EUMETSAT under contracts for the European Commission Copernicus 
             programme.
    License: This code is offered as open source and free-to-use in the public domain, 
             with no warranty, under the MIT license associated with this code repository.
    
This routine has been designed to work with SLSTR L2 NRT data, which is available as tiles. It can be easily adapted for application to L1 data, but care should be taken if using it to analyse L2 NTC data, as this is delivered as a half orbit PDU.

<div class="alert alert-block alert-warning">
    <b>Get the WEkEO User credentials</b>
<hr>
If you want to download the data to use this notebook, you will need WEkEO User credentials. If you do not have these, you can register <a href="https://www.wekeo.eu/web/guest/user-registration" target="_blank">here</a>.


This routine shows examples of how to use python netcdf libraries to ingest Level 2 SLSTR data, mask it according to quality control variables, correct for bias, select only for dual view data and compare it against other, coincident geo-physical variables.

In Python, we usually have two sections of code at the top that occur before we enter the main programme. These
sections typically include:
1. importing modules
2. defining functions that are called in the main programme

Firstly, we begin by importing all of the external modules that we will be using in this script; they are annotated with brief explanations of what they do.

In [ ]:
# import tools that allow us access to system functions, e.g. get working directory, check path.
import os
import shutil
from IPython.core.display import display, HTML
import glob

# import high level python system functions
import sys
import warnings

# import tools that let us manipulate dates and times
import datetime

# import tools that let us manipulate arrays (makes Python more like Matlab for matrix operations)
import numpy as np

# import tools that facilitate string pattern matching
import fnmatch

# import tools that let us create log files to write to
import logging

# import tools for netCDF4 manipulation
import xarray as xr

#import tools for reading and manipulating files
import json
from zipfile import ZipFile

# import tools for plotting, making subplots, and utilising map projections
import matplotlib.pyplot as plt
from matplotlib import gridspec
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# turn off warnings. Bad practice, but we don't want to see them here....
warnings.filterwarnings("ignore")

sys.path.append(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),'wekeo-hda'))
import hda_api_functions as hapi

We will quickly get some map backgrounds for our later plots...

In [ ]:
land_resolution = '50m'
land_poly = cfeature.NaturalEarthFeature('physical', 'land', land_resolution,
                                        edgecolor='k',
                                        facecolor=cfeature.COLORS['land'])

Now we will start our script, proper. First we need to get some data.

WEkEO provides access to a huge number of datasets through its **'harmonised-data-access'** API. This allows us to query the full data catalogue and download data quickly and directly onto the Jupyter Lab. You can search for what data is available <a href="https://wekeo.eu/data?view=catalogue">here</a>

In order to use the HDA-API we need to provide some authentication credentials, which comes in the form of an API key and API token. In this notebook we have provided functions so you can retrieve the API key and token you need directly. You can find out more about this process in the notebook on HDA access (wekeo_harmonized_data_access_api.ipynb) that can be found in the **wekeo-hda** folder on your Jupyterlab.


We will also define a few other parameters including where to download the data to, and if we want the HDA-API functions to be verbose. **Lastly, we will tell the notebook where to find the query we will use to find the data.** These 'JSON' queries are what we use to ask WEkEO for data. They have a very specific form, but allow us quite fine grained control over what data to get. You can find the example one that we will use here: **JSON_templates/EO_EUM_DAT_SENTINEL-3_SL_2_WST___.json**

In [ ]:
# set this key to true to download data.
download_data = False

 # This reduces the resolution of the plot to conserve memory - increasing the number gives a coarser plot
grid_factor = 3 

# your WEkEO API username and password (needs to be in '  ')
user_name = 'USERNAME'
password = 'PASSWORD'

# Generate an API key
api_key = hapi.generate_api_key(user_name, password)
display(HTML('Your API key is: <b>'+api_key+'</b>'))

In [ ]:
# where the data should be downloaded to:
download_dir_path = os.path.join(os.getcwd(),'products')
# where we can find our data query form:
JSON_query_dir = os.path.join(os.getcwd(),'JSON_templates')
# HDA-API loud and noisy?
verbose = False

# make the output directory if required
if not os.path.exists(download_dir_path):
    os.makedirs(download_dir_path)

Now we have set how we want the script to run, we are ready to get some data. We start this process by telling the script what kind of data we want. In this case, this is SLSTR level 2 data, which has the following designation on WEkEO: **EO:EUM:DAT:SENTINEL-3:SL_2_WST___**.

In [ ]:
# SLSTR LEVEL 2 Dataset ID
dataset_id = "EO:EUM:DAT:SENTINEL-3:SL_2_WST___"

Here, we use this dataset_id to find the correct, locally stored JSON query file which describes the data we want. The query file is called: **JSON_templates/EO_EUM_DAT_SENTINEL-3_SL_2_WST___.json**

You can edit this query if you want to get different data, but be aware of asking for too much data - you could be here a while and might run out of space to use this data in the JupyterLab. The box below gets the correct query file.

In [ ]:
# find query file
JSON_query_file = os.path.join(JSON_query_dir,dataset_id.replace(':','_')+".json")
if not os.path.exists(JSON_query_file):
    print('Query file ' + JSON_query_file + ' does not exist')
else:
    print('Found JSON query file for '+dataset_id)

Now we have a query, we need to launch it to WEkEO to get our data. The box below takes care of this through the following steps:
    1. initialise our HDA-API
    2. get an access token for our data
    3. accepts the WEkEO terms and conditions
    4. loads our JSON query into memory
    5. launches our search
    6. waits for our search results
    7. gets our result list
    8. downloads our data

This is quite a complex process, so much of the functionality has been buried 'behind the scenes'. If you want more information, you can check out the **wekeo-hda** tool kit in the parent training directory. The code below will report some information as it runs. At the end, it should tell you that one product has been downloaded.

In [ ]:
if download_data:
    HAPI_dict = hapi.init(dataset_id, api_key, download_dir_path)
    HAPI_dict = hapi.get_access_token(HAPI_dict)
    HAPI_dict = hapi.acceptTandC(HAPI_dict)

    # load the query
    with open(JSON_query_file, 'r') as f:
        query = json.load(f)

    # launch job
    print('Launching job...')
    HAPI_dict = hapi.get_job_id(HAPI_dict, query)

    # check results
    print('Getting results...')
    HAPI_dict = hapi.get_results_list(HAPI_dict)
    HAPI_dict = hapi.get_order_ids(HAPI_dict)

    # download data
    print('Downloading data...')
    HAPI_dict = hapi.download_data(HAPI_dict, file_extension='.zip')

In [ ]:
if download_data:
    # unzip file
    for filename in HAPI_dict['filenames']:
        if os.path.splitext(filename)[-1] == '.zip':
            print('Unzipping file')
            try:
                with ZipFile(filename, 'r') as zipObj:
                    # Extract all the contents of zip file in current directory
                    zipObj.extractall(os.path.dirname(filename))

                # clear up the zip file
                os.remove(filename)
            except:
                print("Failed to unzip....")

In [ ]:
if download_data:
    unzipped_file = HAPI_dict['filenames'][0].replace('.zip','.SEN3')
else:
    unzipped_file = glob.glob(os.path.join(download_dir_path,'*SL_2_WST*.SEN3'))



After all of that preparation, we are at the main entrance point for our code. We begin by defining our logfile so we can record any errors here if things go wrong, or any info/debug messages we may want along the way if we don't want them printed to screen (or to the console in interactive mode).

In [ ]:

DEFAULT_LOG_PATH = os.getcwd()
logfile = os.path.join(DEFAULT_LOG_PATH,"SLSTR_test_plot_"+datetime.datetime.now().strftime('%Y%m%d_%H%M')+".log")
# we define a verbose flag to control how much info we want to see. It can also be useful to define a debug flag
# for even more information.
verbose=False

# this option will stop plotting to screen, and instead plot to file - this can help memory management in the interactive Jupyter environment
no_show=True
if no_show:
    plt.ioff()

We have defined a log file above, and here we set up how python will use it. Note that this is the first time we use the 'print' command. Print will output its contents to the screen, and here, this output will appear below the box when we run it.

In [ ]:
# set file logger
try:
    if os.path.exists(logfile):
        os.remove(logfile)
    print("logging to: "+logfile)
    logging.basicConfig(filename=logfile,level=logging.DEBUG)
except:
    print("Failed to set logger")

So, lets proceed with loading some SLSTR data. The first thing we need to do is find the relevant netCDF files that contain the data that we want to plot. This next block of code collects the names of all netCDF files in our DEFAULT_ROOT_DIR path. We can make this more specific by adapting the DEFAULT_FILE_FILTER variable from "*.nc".

We begin by setting up an empty "list" variables called nc_files, and append to this list as we proceed through a series of loops, defined by the "for" statements.

In [ ]:
# -get the files-------------------------------------------------------------
DEFAULT_FILE_FILTER = '*SLSTR*.nc'
nc_files=[]
for root, _, filenames in os.walk(download_dir_path):
    for filename in fnmatch.filter(filenames, DEFAULT_FILE_FILTER):
        nc_files.append(os.path.join(root, filename))
        if verbose:
            print('Found: '+filename)
        logging.info('Found: '+os.path.join(root, filename))

Lets check what files we have by looping through the list...

In python you can loop through the values in a list using "for item in list:". We only have 1 file in this cacse, though

In [ ]:
for nc_file in nc_files:
    print(nc_file)

For now, we will just work with the first file we find, which is held in nc_files[0].

In [ ]:
nc_file = nc_files[0]

This next line opens our netCDF file. It does not read any data in yet, just makes the contents accessible. We should remember to close this link, especially if we are opening lots of files.

In [ ]:
nc_fid = xr.open_dataset(nc_file)

We start by loading our coordinate variables, using the function that we defined at the top of the script. 

note: python is very accepting of white space, but the next line would flag as a problem in a code-checker like pylint. It is spaced like this to make it easy to read.

In [ ]:
LON  = nc_fid.lon[::grid_factor,::grid_factor]
LAT  = nc_fid.lat[::grid_factor,::grid_factor]
TIME = nc_fid.adi_dtime_from_sst[::grid_factor,::grid_factor]

And finally we load our data variables and close the netCDF file link. SST is stored as a three dimensional variable, with dimensions of time, lat and lon. The time dimension has length of one, and in order to plot SST as an image, we need to remove this "singleton" dimension. The numpy method "squeeze" will take care of this for us.

In [ ]:
SST = np.squeeze(nc_fid.sea_surface_temperature.data[0,::grid_factor,::grid_factor])
SST_STD_DEV = np.squeeze(nc_fid.sses_standard_deviation.data[0,::grid_factor,::grid_factor])
SST_BIAS = np.squeeze(nc_fid.sses_bias.data[0,::grid_factor,::grid_factor])
SST_ALG_TYPE = np.squeeze(nc_fid.sst_algorithm_type.data[0,::grid_factor,::grid_factor])
WIND_SPEED = np.squeeze(nc_fid.wind_speed.data[0,::grid_factor,::grid_factor])

Now we load the quality and masking variables

In [ ]:
L2P_FLAGS     = np.squeeze(nc_fid.l2p_flags.data[0,::grid_factor,::grid_factor])
QUALITY_LEVEL = np.squeeze(nc_fid.quality_level.data[0,::grid_factor,::grid_factor])
nc_fid.close()

Now, lets set up our figure and make an initial plot of our SST data

In [ ]:
fig  = plt.figure(figsize=(6,6), dpi=150)
if no_show:
    plt.imshow(SST)
    fig.savefig('Plot1_Initial.png',bbox_inches='tight')
else:
    plt.imshow(SST)
    plt.show()

Plot successful, but this is ugly, not very helpful as it is not on any geographical map, incorrect as we have not taken abias into account, and has not been quality controlled! Lets take steps to improve this by:
1. reprojecting the data onto a map
2. make a contour plot against our LON and LAT data (we use contourf here as it is faster, but pcolor is more appropriate)
3. apply a more sensible colour bar for SST data
4. adding a colour bar

Then:
1. Masking our data for specific features and qulity values
2. Correcting our the SST for bias and considering the standard deviation
3. Considering dual SST only.

Lastly:
1. Checking associated variables; e.g. wind speed

So, lets perform the steps in the first list. The following code block does exactly that; reprojects (using the basemap toolkit), makes a filled contour plot (using contourf), applies a colour scale (cmap) and adds a colour bar (plt.colorbar).

In [ ]:
fig  = plt.figure(figsize=(10,10), dpi=300)

# set projection
m = plt.axes(projection=ccrs.PlateCarree(central_longitude=0.0))

# set my vertical plotting order and fontsize
zordcoast=0
fsz=12
SST_plot = SST.copy()
vmin=np.nanmean(SST_plot)-3*np.nanstd(SST_plot)
vmax=np.nanmean(SST_plot)+3*np.nanstd(SST_plot)
SST_plot[SST_plot<vmin] = np.nan
SST_plot[SST_plot>vmax] = np.nan

# plot the data
p1 = plt.pcolormesh(LON,LAT,SST_plot,cmap=plt.cm.jet,vmin=vmin,vmax=vmax)

# add embelishments
m.coastlines(resolution=land_resolution, color='black', linewidth=1)
m.add_feature(land_poly)
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.xlabel_style = {'size': 16, 'color': 'gray'}
g1.ylabel_style = {'size': 16, 'color': 'gray'}

cbar = plt.colorbar(p1, orientation='horizontal')
cbar.set_label('SST [K]',fontsize=fsz);
plt.title('SLSTR SST [K]', fontsize=fsz);

In [ ]:
if no_show:
    fig.savefig('Plot2_SST.png',bbox_inches='tight')
else:
    plt.show()

A definite improvement, our data is now accompanied by a scale, with units, and is reprojected so we can relate it to a map.

However, we still have not interrogated our data. 

One of the most important things we need to do with SST data is consider the quality level - so lets start by doing that. 

The next code bloack will display the values of the quality level, stored in the quality level variables in the L2 WST product. Usually, we only consider the product viable where the quality flag is five, but can use quality level 4 in some circumstances.

In [ ]:
fig  = plt.figure(figsize=(10*int(np.nanmax(QUALITY_LEVEL))+1,10), dpi=150)
gs  = gridspec.GridSpec(1, int(np.nanmax(QUALITY_LEVEL))+1)
contour_vals = np.arange(np.nanmin(QUALITY_LEVEL)-1,np.nanmax(QUALITY_LEVEL)+1,1)
gs.update(wspace=0.1, hspace=0.1)
# loop through each algorithm
for ii in np.arange(0,int(np.nanmax(QUALITY_LEVEL))+1):
    m = plt.subplot(gs[0,ii], projection=ccrs.PlateCarree(central_longitude=0.0))
    MASKED_QUALITY_LEVEL = QUALITY_LEVEL.astype('float')
    MASKED_QUALITY_LEVEL[MASKED_QUALITY_LEVEL != float(ii)] = np.nan

    # plot the data
    plt.pcolormesh(LON,LAT,np.ma.masked_invalid(MASKED_QUALITY_LEVEL),vmin=0,vmax=5)
    plt.text(np.nanmin(LON),np.nanmin(LAT),ii,fontweight='bold',fontsize=fsz*2)

    # add embelishments
    m.coastlines(resolution=land_resolution, color='black', linewidth=1)
    m.add_feature(land_poly)
    g1 = m.gridlines(draw_labels = False)
    g1.xlabel_style = {'size': 16, 'color': 'gray'}
    g1.ylabel_style = {'size': 16, 'color': 'gray'}

In [ ]:
if no_show:
    fig.savefig('Plot3_Quality.png',bbox_inches='tight')
else:
    plt.show()

So, lets mask out any data that have a quality value of 2 or lower...

In [ ]:
SST[QUALITY_LEVEL<=4]=np.nan

Next, the SST field has two associated measurements that we need to consider, the bias, and the standard deviation. So lets plot these...

In [ ]:
fig  = plt.figure(figsize=(20,20), dpi=300)
gs  = gridspec.GridSpec(1, 2)
fsz = 20

m = plt.subplot(gs[0,0], projection=ccrs.PlateCarree(central_longitude=0.0))

# plot the data
SST_plot = SST_BIAS.copy()
vmin=np.nanmean(SST_plot)-3*np.nanstd(SST_plot)
vmax=np.nanmean(SST_plot)+3*np.nanstd(SST_plot)
SST_plot[SST_plot<vmin] = np.nan
SST_plot[SST_plot>vmax] = np.nan

p1 = plt.pcolormesh(LON,LAT,SST_plot,cmap=plt.cm.jet,vmin=vmin,vmax=vmax)
# add embelishments
m.coastlines(resolution=land_resolution, color='black', linewidth=1)
m.add_feature(land_poly)
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.ylabels_right = False
g1.xlabel_style = {'size': 16, 'color': 'gray'}
g1.ylabel_style = {'size': 16, 'color': 'gray'}

cbar = plt.colorbar(p1, orientation='horizontal', pad=0.05)
cbar.set_label('SST bias [K]',fontsize=fsz)

m = plt.subplot(gs[0,1], projection=ccrs.PlateCarree(central_longitude=0.0))
# plot the data
SST_plot = SST_STD_DEV.copy()
vmin=np.nanmean(SST_plot)-3*np.nanstd(SST_plot)
vmax=np.nanmean(SST_plot)+3*np.nanstd(SST_plot)
SST_plot[SST_plot<vmin] = np.nan
SST_plot[SST_plot>vmax] = np.nan

p1 = plt.pcolormesh(LON,LAT,SST_plot,cmap=plt.cm.jet,vmin=vmin,vmax=vmax)
# add embelishments
m.coastlines(resolution=land_resolution, color='black', linewidth=1)
m.add_feature(land_poly)
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.ylabels_right = False
g1.xlabel_style = {'size': 16, 'color': 'gray'}
g1.ylabel_style = {'size': 16, 'color': 'gray'}

cbar = plt.colorbar(p1, orientation='horizontal', pad=0.05)
cbar.set_label('SST standard deviation [K]',fontsize=fsz);

In [ ]:
if no_show:
    fig.savefig('Plot4_SSES.png',bbox_inches='tight')
else:
    plt.show()

The SST value we are interested in needs to be corrected for the bias, so lets do that:

In [ ]:
SST = SST + SST_BIAS

We can see from the bias and standard deviation plots that there are some sharp lines across the image. SLSTR uses 5 algorithms to estimate SST. Some of these algorithms use the nadir view only, while some take advantage of the 'Dual View' capability of the sensor. Dual view takes two images of the surface, one at nadir and one at an oblique angle. This allows it to better characterise the effects of the atmosphere. In practice, the nadir view is wider than the dual view, which results in a stripe along the middle of the swath. We can check which algorithms were used to derive the SST estimate by checking the SST_ALG_TYPE variable, as below:

In [ ]:
fig  = plt.figure(figsize=(10*int(np.nanmax(SST_ALG_TYPE))+1,10), dpi=150)
gs  = gridspec.GridSpec(1, 6)
gs.update(wspace=0.1, hspace=0.1)
fsz = 12

contour_vals = np.arange(np.nanmin(SST_ALG_TYPE)-1,np.nanmin(SST_ALG_TYPE)+1,1)

# loop through each algorithm
for ii in np.arange(0,int(np.nanmax(SST_ALG_TYPE))+1):
    m = plt.subplot(gs[0,ii], projection=ccrs.PlateCarree(central_longitude=0.0))
    MASKED_ALG_TYPE = SST_ALG_TYPE.astype('float')
    MASKED_ALG_TYPE[MASKED_ALG_TYPE != float(ii)] = np.nan
    # plot the data
    plt.pcolormesh(LON,LAT,np.ma.masked_invalid(MASKED_ALG_TYPE),cmap=plt.cm.jet,vmin=0,vmax=5)
    if ii == 0:
        plt.text(np.nanmin(LON),np.nanmin(LAT),'No retrieval',fontweight='bold',fontsize=fsz*2)
    if ii == 1:
        plt.text(np.nanmin(LON),np.nanmin(LAT),'N2',fontweight='bold',fontsize=fsz*2)
    if ii == 2:
        plt.text(np.nanmin(LON),np.nanmin(LAT),'N3R',fontweight='bold',fontsize=fsz*2)
    if ii == 3:
        plt.text(np.nanmin(LON),np.nanmin(LAT),'N3',fontweight='bold',fontsize=fsz*2)
    elif ii == 4:
        plt.text(np.nanmin(LON),np.nanmin(LAT),'D2',fontweight='bold',fontsize=fsz*2)
    elif ii == 5:
        plt.text(np.nanmin(LON),np.nanmin(LAT),'D3',fontweight='bold',fontsize=fsz*2)

    # add embelishments
    m.coastlines(resolution=land_resolution, color='black', linewidth=1)
    m.add_feature(land_poly)
    g1 = m.gridlines(draw_labels = False)
    g1.xlabel_style = {'size': 16, 'color': 'gray'}
    g1.ylabel_style = {'size': 16, 'color': 'gray'}

In [ ]:
if no_show:
    fig.savefig('Plot5_Algorithms.png',bbox_inches='tight')
else:
    plt.show()

We should remember that, just because a measurement is Nadir view only, it does not mean that it is bad! Sometimes the nadir view is the best to use. Here, though, lets finally plot our nadir+dual and our dual view data, corrected for bias, and masked for a quality level of 3 or greater. We will overlay the plot with contours from contemporaneous ECMWF wind data, that is included in with SLSTR L2 WAT products, as part of the GHRSST specification. The final plot is saved in the directory where this code is stored. You can open it by double clicking the file name on the left. 

In [ ]:
%%capture
SST_C = SST-273.15

fig  = plt.figure(figsize=(20,20), dpi=150)
gs  = gridspec.GridSpec(3, 1, height_ratios=[20,0.5,1])
gs.update(wspace=0.01, hspace=0.01)

# set my vertical plotting order and fontsize
zordcoast=0
fsz=20

# plot the data
SST_plot = SST_C.copy()
vmin=int(np.nanmean(SST_plot)-3*np.nanstd(SST_plot))-1
vmax=int(np.nanmean(SST_plot)+3*np.nanstd(SST_plot))+1
SST_plot[SST_plot<vmin] = np.nan
SST_plot[SST_plot>vmax] = np.nan

m = plt.subplot(gs[0,0], projection=ccrs.PlateCarree(central_longitude=0.0))
p1 = plt.contourf(LON,LAT,SST_plot,100,cmap=plt.cm.jet,vmin=vmin,vmax=vmax,zorder=-1)
CS = plt.contour(LON,LAT,WIND_SPEED,10,linewidths=1.0,cmap=plt.get_cmap('Greys'),zorder=0)
plt.clabel(CS, fontsize=10, inline=1)

# add embelishments
m.coastlines(resolution=land_resolution, color='black', linewidth=1)
m.add_feature(land_poly)
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.xlabel_style = {'size': 16, 'color': 'gray'}
g1.ylabel_style = {'size': 16, 'color': 'gray'}

# add colorbar
axes0 = plt.subplot(gs[2,0])
cbar = plt.colorbar(p1, cax=axes0, orientation='horizontal')
cbar.ax.tick_params(labelsize=fsz) 
cbar.set_label('Bias corrected, quality controlled, whole view SST [$^{o}$C]',fontsize=fsz)

In [ ]:
bbox_inches='tight'
fig.savefig('Plot6_SLSTR_All_SST.png',bbox_inches='tight')

In [ ]:
%%capture

SST_C = SST-273.15
SST_C[SST_ALG_TYPE<4] = np.nan

fig  = plt.figure(figsize=(20,20), dpi=150)
gs  = gridspec.GridSpec(3, 1, height_ratios=[20,1,1])
gs.update(wspace=0.01, hspace=0.01)  

# set my vertical plotting order and fontsize
zordcoast=0
fsz=20

# plot the data
SST_plot = SST_C.copy()
vmin=int(np.nanmean(SST_plot)-3*np.nanstd(SST_plot))-1
vmax=int(np.nanmean(SST_plot)+3*np.nanstd(SST_plot))+1
SST_plot[SST_plot<vmin] = np.nan
SST_plot[SST_plot>vmax] = np.nan

m = plt.subplot(gs[0,0], projection=ccrs.PlateCarree(central_longitude=0.0))
p1 = m.contourf(LON,LAT,SST_plot,100,cmap=plt.cm.jet,vmin=vmin,vmax=vmax,zorder=-1)
CS = m.contour(LON,LAT,WIND_SPEED,10,linewidths=1.0,cmap=plt.get_cmap('Greys'),zorder=0)
plt.clabel(CS, fontsize=14, inline=1)

# add embelishments
m.coastlines(resolution=land_resolution, color='black', zorder=3)
m.add_feature(land_poly, zorder = 2)
g1 = m.gridlines(draw_labels = True)
g1.xlabels_top = False
g1.xlabel_style = {'size': 16, 'color': 'gray'}
g1.ylabel_style = {'size': 16, 'color': 'gray'}

# add colorbar
axes0 = plt.subplot(gs[2,0])
cbar = plt.colorbar(p1, cax=axes0, orientation='horizontal')
cbar.ax.tick_params(labelsize=fsz) 
cbar.set_label('Bias corrected, quality controlled, dual view SST [$^{o}$C]',fontsize=fsz)

In [ ]:
bbox_inches='tight'
fig.savefig('Plot7_SLSTR_Dual_SST_demo.png',bbox_inches='tight')

<img src='./img/all_partners_wekeo.png' alt='' align='center' width='75%'></img>

<p style="text-align:left;">This project is licensed under the <a href="./LICENSE">MIT License</a> <span style="float:right;"><a href="https://github.com/wekeo/wekeo-jupyter-lab">View on GitHub</a> | <a href="https://www.wekeo.eu/">WEkEO Website</a> | <a href=mailto:support@wekeo.eu>Contact</a></span></p>